# read csv

In [21]:
import pandas as pd
import os 

In [121]:
dfs=[]
for root, dirs, fnames in os.walk('scapy_output/'):
        for fname in fnames:
            if '.csv' in fname:
                df=pd.read_csv(os.path.join(root,fname))
                dfs.append(df)

In [124]:
len(dfs)



30

In [141]:
def get_features(test):
    rows=[]
    for row in test.iterrows():
        row=row[1]
        hp=row['highest protocal']
        l4p=row['L4 Protocal']
        protocol=['1']
        if pd.isna(l4p):
            dport='0'
            protocol='0-0-0-0-0-0-1'
            dir='2'
        else:
            dport=row['dst port']

            if 'TCP' in hp or 'TCP' in l4p:
                protocol.append('1')
            else:
                protocol.append('0')
            if 'UDP' in hp or 'UDP' in l4p:
                protocol.append('1')
            else:
                protocol.append('0')
            if 'TLS' in hp or 'TLS' in l4p or 'SSL' in hp or 'SSL' in l4p:
                protocol.append('1')
            else:
                protocol.append('0')
            if 'HTTP' in hp or 'HTTP' in l4p:
                protocol.append('1')
            else:
                protocol.append('0')
            if 'DNS' in hp or 'DNS' in l4p:
                protocol.append('1')
            else:
                protocol.append('0')
            protocol.append('0')
            protocol='-'.join(protocol)

            if row['src ip'][:7]=='192.168':
                dir=1
            else:
                dir=0

        packet_size=row['pkt len']
        time=row['time']     
        rows.append([dport,protocol,dir,packet_size,time])
    return pd.DataFrame(rows,columns=['dport','protocol','dir','packet_size','time'])


In [145]:
results=[]
for df,name in zip(dfs,fnames):
    result=get_features(df)
    result.time=result.time.diff()
    result=pd.concat([result.drop(columns='protocol'),result.protocol.str.split('-',expand=True)],axis=1)
    results.append(result)
    result.dropna().to_csv('final_output/{}'.format(name),index=False)
